## Data Loading and Image Preprocessing

Load Train Data and Basic Preprocessing - mask, blur, segment and sharpen

In [1]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
import cv2

In [2]:
scaleSize = 299  #resize figures to 
seed = 0        #random seed

In [3]:
# Covert ClassName to Integer method
def label_s2n(x):
    return {
        'Black-grass': 0,
        'Charlock': 1,
        'Cleavers': 2,
        'Common Chickweed': 3,
        'Common wheat': 4, 
        'Fat Hen': 5,
        'Loose Silky-bent': 6,
        'Maize': 7,
        'Scentless Mayweed': 8,
        'Shepherds Purse': 9,
        'Small-flowered Cranesbill': 10,
        'Sugar beet': 11
    }.get(x, 12)

def label_n2s(x):
    return {
        0: 'Black-grass',
        1: 'Charlock',
        2: 'Cleavers',
        3: 'Common Chickweed',
        4: 'Common wheat', 
        5: 'Fat Hen',
        6: 'Loose Silky-bent',
        7: 'Maize',
        8: 'Scentless Mayweed',
        9: 'Shepherds Purse',
        10: 'Small-flowered Cranesbill',
        11: 'Sugar beet'
    }.get(x, 'None')

In [5]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
             'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']

path='Dataset/train/*/*.png'
files = glob(path)

trainImage = []
trainLabel = []
j = 1
num = len(files)

for img in files:
    if (j >= num):
        print(str(j)+"/"+str(num), end="\r")
    trainImage.append(cv2.resize(cv2.imread(img), (scaleSize, scaleSize)))
    trainLabel.append(img.split('/')[-2])   #Depends on the path of dataset, MODIFY
    j = j + 1
    
trainImage = np.asarray(trainImage)
trainLabel = pd.DataFrame(trainLabel)

In [6]:
# Covert Categories into Label Numbers
trainLabels = []
for i in range(trainLabel.shape[0]):
    trainLabels.append(label_s2n(trainLabel[0][i]))

In [7]:
np.save(os.path.join(os.getcwd(),'trainImage.npy'), trainImage)
np.save(os.path.join(os.getcwd(),'trainLabels.npy'), trainLabels)

In [8]:
clearTrainImage = []
maskTrainImage = []
blurredTrainImage = []
segmentTrainImage = []
sharpTrainImage = []
examples = []
getEx = True
for image in trainImage:
    
    #Convert to HSV
    hsvImage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #Create mask
    sensitivity = 35
    green_lb = (60 - sensitivity, 100, 50) #lower bound of green range
    green_ub = (60 + sensitivity, 255, 255) #upper bound of green range
    mask = cv2.inRange(hsvImage, green_lb, green_ub)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    #segment image
    segmentImage = cv2.bitwise_and(image, image, mask = mask)
    
    #sharpen
    blurredImage = cv2.GaussianBlur(segmentImage, (5,5), 0)
    sharpImage = cv2.addWeighted(segmentImage, 1.5, blurredImage, -0.5, 0)
    maskTrainImage.append(mask)
    segmentTrainImage.append(segmentImage)
    blurredTrainImage.append(blurredImage)
    sharpTrainImage.append(sharpImage)

In [9]:
np.save(os.path.join(os.getcwd(),'maskTrainImage.npy'), maskTrainImage)
np.save(os.path.join(os.getcwd(),'segmentTrainImage.npy'), segmentTrainImage)
np.save(os.path.join(os.getcwd(),'blurredTrainImage.npy'), blurredTrainImage)
np.save(os.path.join(os.getcwd(),'sharpTrainImage.npy'), sharpTrainImage)

In [10]:
path='Dataset/test/*.png'
files = glob(path)

testImage = []

j = 1
num = len(files)

for img in files:
    if (j >= num):
        print(str(j)+"/"+str(num), end="\r")
    testImage.append(cv2.resize(cv2.imread(img), (scaleSize, scaleSize)))
    j = j + 1
    
testImage = np.asarray(testImage)

In [11]:
np.save(os.path.join(os.getcwd(),'testImage.npy'), testImage)

In [12]:
clearTestImage = []
maskTestImage = []
blurredTestImage = []
segmentTestImage = []
sharpTestImage = []
examples = []
getEx = True
for image in testImage:
    
    #Convert to HSV
    hsvImage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #Create mask
    sensitivity = 35
    green_lb = (60 - sensitivity, 100, 50) #lower bound of green range
    green_ub = (60 + sensitivity, 255, 255) #upper bound of green range
    mask = cv2.inRange(hsvImage, green_lb, green_ub)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    #segment image
    segmentImage = cv2.bitwise_and(image, image, mask = mask)
    
    #sharpen
    blurredImage = cv2.GaussianBlur(segmentImage, (5,5), 0)
    sharpImage = cv2.addWeighted(segmentImage, 1.5, blurredImage, -0.5, 0)
    maskTestImage.append(mask)
    segmentTestImage.append(segmentImage)
    blurredTestImage.append(blurredImage)
    sharpTestImage.append(sharpImage)

In [13]:
np.save(os.path.join(os.getcwd(),'sharpTestImage.npy'), sharpTestImage)